In [2]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [8]:
def process(data):
    p = data.text.split(',')[0]
    datum = {'name': p,
             'latitude': None,
             'longitude': None}
    try:
        l = 'https://en.wikipedia.org' + data.contents[0]['href']
        next_page = requests.get(l)
        next_content = BeautifulSoup(next_page.content, 'lxml')
        lat_long = next_content.find('a', class_='external text').attrs['href']
        try:
            coord_page = requests.get(lat_long)
        except:
            lat_long = 'http:' + lat_long
            coord_page = requests.get(lat_long)
        
            coord_content = BeautifulSoup(coord_page.content, 'lxml')
            lat = coord_content.find('span', class_='latitude p-latitude')
            long = coord_content.find('span', class_='longitude p-longitude')
            datum = {'name': p,
                     'latitude': lat.text if lat else None,
                     'longitude': long.text if long else None}
    except:
        pass
    
    return datum

In [9]:
url = 'https://en.wikipedia.org/wiki/Category:Census-designated_places_in_Los_Angeles_County,_California'
page = requests.get(url)
content = BeautifulSoup(page.content, 'lxml')
temp = content.find_all('div', class_='mw-category')[-1]
eph = temp.find_all('li')
data = []
for i in tqdm(eph):
    datum = process(i)
    data.append(datum)
df = pd.DataFrame(data)
df.to_csv('./data/la_places.csv')

100%|███████████████████████████████████████████| 53/53 [00:49<00:00,  1.08it/s]
